<h1 style="color:#333333; text-align:center; line-height: 0;"> <img style="right;" src="logo.png" width=18% height=18%> Reinforcement Learning | Assignment 3 
</h1>
<br/><br/>


The goal of this assignment is to implement:
- Critic (Model and critic itself)
- Actor
- PyTorch optimizer
- Actor-Critic algorithm scenario

___Total points:___ 100

In [ ]:
!pwd

###  <font color="blue"> A brief introduction </font>
Examine it carefully, it covers most of your possible needs to make an assignment.

***

### About Rcognita
The platform for this (and all subsequent work) is [Rcognita](https://gitflic.ru/project/aidynamicaction/rcognita), a framework for applying control theory and machine learning algorithms to control problems, an integral part of which is the closed-loop interaction between the agent under control and the environment evolving over time. In the Rcognita paradigm, the main bearer of all the classes and variables needed to run the simulation is the `pipeline`. 

The main parts of `pipeline` are: 
* `simulator`, which is defined at module `simulators.py` and responsible for simulation of evolution of the environment
* `actor`, defined at module `actors.py`, which is responsible for obtaining of action
* `critic`, defined at module `critics.py`, which is reponsible for learning of reward function and obtaining its value 
* `controller`, which is defined at module `controllers.py` and it's needed to put it all together into an RL (or other) controller
* `system`, which is defined at module `systems.py`.

Other minor things are also declarated in the pipeline and assembled module by module up to the execution of the pipeline itself. 
Just to be on the same page, we provide some notation to prevent further confusions.
* `weights` is the general name and for weights of neural network and for values in tables of value function and policy as well. This agreement comes from the motivation for being consistent with classical RL where critic and actor are being implemented as some neural networks with some **weights**. So, here comes the second term
* `model`. It's obvious that parameters give specificity to something. But the general form itself is being called `model`. There are plenty of models of different types and forms (such as NN). Model is what critic and actor and even running cost always have, no matter what.
* `predictor` - Inspite of it's cryptic name, this object performs an important function, namely, it carries the law by which the dynamics of our system is being predicted in future. For example, if we have some differential equation
$
\begin{cases}
\dot{\boldsymbol x} = \boldsymbol f(\boldsymbol x, \boldsymbol u)\\
\boldsymbol y = h(\boldsymbol x) \\
\boldsymbol x(0)=\boldsymbol x_{0}\\
\end{cases}
$
where $x_{0}$ is the **initial state**.
in general, there are several ways of prediction: 
> - **Analytical**, when we have a precise formula of analytical solution $\boldsymbol x(t)$ to the ODE and have no problems to compute it at any given time. This is great but not that possible in real life. Nevertheless, our predictor could be expressed like:  $\text{predictor}(\boldsymbol x(\tau),dt) = \boldsymbol x(\tau + dt)$
> - **Numerical** way is mostly a case. The simplest way of prediction then is an Euler method:
$\boldsymbol x_{k+1}= \text{predictor}(\boldsymbol x_k, \delta)=\boldsymbol x_{k}+\delta \boldsymbol f\left(\boldsymbol x_{k}, \boldsymbol u_{k}\right) \text {, }$

In this assignment we meet a new object - **scenario**. Scenario is a module that forms and executes the main loops for different scenarios, like online or episodical scenario. In this assignment we will use an episodical scenario.


<a id='Notation'></a>
### Notation summary
From now and on we will use the following notation:

| Notation &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| &nbsp;&nbsp;Description |
|:-----------------------:|-------------|
| $\boldsymbol f(\cdot, \cdot, \cdot) : \mathbb{R}^{n+1}\times \mathbb{R}^{m} \rightarrow \mathbb{R}^{n}$ |A **state dynamic function** or, more informally, **righ-hand-side** of a system <br /> of ordinary differential equations $\dot{\boldsymbol x} = \boldsymbol f(t, \boldsymbol x, \boldsymbol u)$|
| $\boldsymbol x \in \mathbb{R}^{n} $ | An element of the **state space** of a controlled system of dimensionality $n$ |
| $\boldsymbol u \in \mathbb{R}^{m}$ | An element of the **action space** of a controlled system of dimensionality $m$ |
| $\boldsymbol y \in \mathbb{R}^{k}$ | An **observartion**|
| $\mathbb{X}\subset \mathbb{R}^{n} $| **State constraint set**|
| $\mathbb{U}\subset \mathbb{R}^{m} $| **Action constraint set**|
| $\boldsymbol h(\cdot): \mathbb{R}^{n} \rightarrow \mathbb{R}^{k}$ | **Observation function**  |
| $\boldsymbol\rho(\cdot) : \mathbb{R}^{n} \rightarrow \mathbb{R}^{m}$ | **Policy** function |
| $r(\cdot) : \mathbb{R}^n \times \mathbb{R}^m \rightarrow \mathbb{R}$ | **Running cost** function  |


### Goal
Our main goal here is to implement Actor-Critic algorithm to [PID-regulator](https://en.wikipedia.org/wiki/PID_controller) coefficients tuning

###  <font color="blue"> Algorithm description </font>

In this setup we will interchange a trivial critic with the "action-value" one. The purpose of this critic is to learn the $Q$-function, so the algorithm will be constructed as follows:

I. **Initialization**:
- set iterations number **N_iterations**
- set episodes number **N_episodes**
- set **discount factor** $\gamma$
- initialize some **policy** parameters $\boldsymbol w_0$, learning rate $\eta$
- initialize some **critic** parameters $\boldsymbol \vartheta_0$, learning rate $\hat{\eta}$

II. **Main loop**:<br/>
(Run episodical scenario)
>**for** i in range(**N_iterations**):
>>**for** j in range(**N_episodes**):
>>> **while** **time** < **t1**:
(corresponding utilized parts of Rcognita are provided in bold inside parentheses)
>>>> - simulate environment evolution (**simulator**, **system**)
>>>> - obtain observation (**system**) $\boldsymbol y_i = \boldsymbol h(\boldsymbol x_i)$
>>>> - obtain action (**actor**) $u_i \sim  \mathcal{N}(\mu,\,\sigma^{2})$
>>>> - compute and store new gradient (**actor.model**)
>>>  - **reset episode:**
>>>> - compute and store REINFORCE objective gradient (**scenario**): $\sum_{k=0}^N \nabla_w \ln \rho^w(\boldsymbol u_k \vert \boldsymbol y_k) \cdot Q\left(\boldsymbol y_k, \boldsymbol u_k\right)$
>>>> - compute and store sum of squared Temporal Difference terms (scenario): $\sum_{k=0}^N \text{TD}_k^2$
>> - **iteration update:**
>>> - **critic update**
>>>> - compute **mean**(`squared_TD_sums_of_episodes`) = $\mathbb{E}[\sum_{k=0}^N \text{TD}_k^2]$ - mean of saquared TD by episodes
>>>> - $\boldsymbol \vartheta_{i+1} = \boldsymbol \vartheta_{i} - \hat{\eta}\nabla_{\boldsymbol \vartheta} \mathbb{E}[\sum_{k=0}^N \text{TD}_k^2]$ (1 step)
>>> - **actor update**
>>>> - compute **mean** overall stored REINFORCE objective gradients (**scenario**): $\mathbb{E}\left[\sum_{k=0}^N \nabla_{\boldsymbol w} \ln \rho^{\boldsymbol w}(\boldsymbol u_k \vert \boldsymbol y_k) \cdot Q\left(\boldsymbol y_k, \boldsymbol u_k\right)\right]$
>>>> - perform a gradient step (**scenario**): $\boldsymbol w_{i+1}=\boldsymbol w_i+\eta \mathbb{E}\left[\sum_{k=0}^N \nabla_{\boldsymbol w} \ln \rho^{\boldsymbol w}(\boldsymbol u_k \vert \boldsymbol y_k) \cdot Q\left(\boldsymbol y_k, \boldsymbol u_k\right)\right]$
>> - **reset iteration** ...

***

In [150]:
%%capture
"""
Just importing all the necessary stuff here.
DO NOT CHANGE
"""
%matplotlib qt
%load_ext autoreload
%autoreload 2

from rcognita_framework.pipelines.pipeline_inverted_pendulum_AC_async import PipelineInvertedPendulumAC
from rcognita_framework.rcognita.actors import ActorProbabilisticEpisodic
from rcognita_framework.rcognita.critics import CriticActionValue
from rcognita_framework.rcognita.systems import SysInvertedPendulum
from rcognita_framework.rcognita.models import ModelGaussianConditional, ModelNN
from rcognita_framework.rcognita.scenarios import EpisodicScenario
from rcognita_framework.rcognita import models
from rcognita_framework.rcognita.utilities import rc
from rcognita_framework.rcognita.optimizers import BaseOptimizer
import numpy as np
from torch import nn
import torch
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import warnings

<h2 style="color:#A7BD3F;"> Section 1: Critic implementation </h2>

***
Contents:
* Model implementation
* Critic implementation

***

Implement your topology here. You can try out pass `(input tensor ** 2)` into linear layer. Why is that? Because one may observe that critic should be semi-negative-definite in our case. In other words, `critic([0,0,0],[0])` should be zero and `critic(y, u)` < 0

In [151]:
class ModelNNStudent(ModelNN):

    model_name = "NN"

    def __init__(self, dim_observation, dim_action, *args, weights = None, **kwargs):
        super().__init__() # dim_observation, dim_action, *args, weights=weights, **kwargs
        
        
        #############################################
        # YOUR CODE BELOW
        #############################################

        self.linear_1 = nn.Linear(dim_observation + dim_action, (dim_observation + dim_action)*2)
        self.linear_2 = nn.Linear((dim_observation + dim_action)*2, (dim_observation + dim_action)*2)
        self.linear_3 = nn.Linear((dim_observation + dim_action)*2, (dim_observation + dim_action)*2)
        self.linear_4 = nn.Linear((dim_observation + dim_action)*2, 1)
#         print('dim_observation', dim_observation, 'dim_action', dim_action)
            
        #############################################
        # YOUR CODE ABOVE
        #############################################
        
        if weights is not None:
            self.load_state_dict(weights)
        self.double()
        self.cache_weights()

    def forward(self, input_tensor, weights=None):
        if weights is not None:
            self.update(weights)
        
        #############################################
        # YOUR CODE BELOW
        #############################################
#         print('input_tensor_ModelNNStudent', input_tensor)
        x = self.linear_1(input_tensor).relu()
        x = self.linear_2(x).relu()
        x = self.linear_3(x).relu()
        x = self.linear_4(x).relu()
        x = torch.linalg.vector_norm(x)
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

        return x


### Q-critic (Action-Value-critic) implementation

Here we will implement a temporal difference.
FYI:
* Vectores in the data buffer are stored in the order from the latest (top, vectors indiced as `[0,:]`) to the  recent one (bottom, `[-1, :]`)
* Data buffer length corresponds to `self.data_buffer_size`
* Important reminder: all models have an ability to cache themselves. Here you can use this to evaluate a TD correctly. Just set `use_stored_weights=True` when you call the critic's model. It will invoke cached weights that also **were detached automatically**. So, you can construct a temporal difference without using Torch functionality  directly. Here are some examples:

In [152]:
model = ModelNNStudent(3, 1)
model([1.,2.,3.], [1.])

tensor(0.1104, dtype=torch.float64, grad_fn=<LinalgVectorNormBackward0>)

In [153]:
model([1.,2.,3.], [1.], use_stored_weights=True) #### gradient won't flow through this tensor

tensor(0.1104, dtype=torch.float64)

Note, how the first input is different from the second one. Okay, let's move on!
* Last, but not least! $\text{TD}(\boldsymbol y_{\text{old}},y_{\text{next}},a_{\text{old}},a_{\text{next}})= \text{critic}(y_{\text{old}}, a_{\text{old}}) - \text{critic}^*(y_{\text{next}}, a_{\text{next}})$, where $\text{critic}^*$ is a critic with fixed weights!.

* Function `objective` should return sum of all possible squared TDs given the current data buffer

In [154]:
class CriticActionValueStudent(CriticActionValue):
    def objective(self, data_buffer=None, weights=None):
        """
        Objective of the critic, say, a squared temporal difference.

        """
        if data_buffer is None:
            observation_buffer = self.observation_buffer
            action_buffer = self.action_buffer
        else:
            observation_buffer = data_buffer["observation_buffer"]
            action_buffer = data_buffer["action_buffer"]

        critic_objective = 0
        
        ####### At this point the data buffer is available, just use it #######
        
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        for i in range(self.data_buffer_size - 1, -1, -1):
            c = self.model(observation_buffer[i - 1, :], action_buffer[i - 1, :], weights=weights)
            n = self.model(observation_buffer[i, :], action_buffer[i, :], use_stored_weights=True)
            out = (c - self.discount_factor * n - self.running_objective(observation_buffer[i - 1, :], action_buffer[i - 1, :]))
            critic_objective += out ** 2 / 2
            
        #############################################
        # YOUR CODE ABOVE
        #############################################

        return critic_objective

<h2 style="color:#A7BD3F;"> Section 2: Actor modification </h2>

As you might remember from the last assignment, you've implemented the evaluation of the policy distribution gradient in the `update` Actor's method. Now the only thing we should do is change it a bit. 

Take the `update` function from your last solution, insert it here and multiply your gradient by Q-function using your critic.


In [155]:
class ActorProbabilisticEpisodicACStudent(ActorProbabilisticEpisodic):
    def update(self, observation):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        action = self.model.sample_from_distribution(observation)
        self.action = np.array(np.clip(action, self.action_bounds[0], self.action_bounds[1]))
        self.action_old = self.action
        gradient = self.model.compute_gradient(action)
        self.store_gradient(gradient) 
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

<h2 style="color:#A7BD3F;"> Section 3: Optimizer implementation </h2>

Torch optimization loop should be implemented here.
1. Zero all gradients
2. Compute loss with objective function you've passed
3. Invoke gradients evaluation with .backward()
4. perform optimization step with .step()

If you have no idea what is going on here, read [this](https://pytorch.org/docs/stable/optim.html) please (or DM @odinmaniac in Telegram)

We need torch here just to perform a gradient step. And literally one step, **no more**! (if you don't understand why - examine code and algorithm carefully because you don't understand it fully)

In [156]:
class TorchOptimizerStudent(BaseOptimizer):
    engine = "Torch"

    def __init__(
        self, opt_options, iterations=1, opt_method=torch.optim.Adam, verbose=False
    ):
        self.opt_method = opt_method
        self.opt_options = opt_options
        self.iterations = iterations
        self.verbose = verbose
        self.loss_history = []

    def optimize(self, objective, model, objective_input):
        optimizer = self.opt_method(
            model.parameters(), **self.opt_options, weight_decay=0
        )
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        for i in range(self.iterations):
            optimizer.zero_grad()
            loss = objective(objective_input)
            loss.backward()
            optimizer.step()
        self.loss_history.append(objective(objective_input))     
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

<h2 style="color:#A7BD3F;"> Section 4: Scenario implementation</h2>

***

In this task you will just modify the scenario you've touched in the previous assignment with adding some corrections related to critic updating. This is an elementary task that is needed just to make you think about what is happening at the end of episode and iteration. 
* At the end of the episode you should append a critic objective value to `squared_TD_sums_of_episodes` list
* in the iteration update phase you launch optimizer with `optimizer.optimize`
* when you reset the iteration, you just empty `squared_TD_sums_of_episodes`


In [157]:
class EpisodicScenarioAsyncACStudent(EpisodicScenario):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.critic_optimizer = TorchOptimizerStudent({"lr": 0.01})
        self.squared_TD_sums_of_episodes = []
        self.square_TD_means = []

    def reset_episode(self):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        self.squared_TD_sums_of_episodes.append(self.critic.objective())
        
        #############################################
        # YOUR CODE ABOVE
        #############################################
        super().reset_episode()

    def iteration_update(self):
        """
        Proposition: create a lambda function that evaluates mean 
        to pass this function as an objective into your optimizer.
        Keep in mind that you must not detach your tensors with inplace somehow.
        If you call, for example, torch.tensor(tens), it will detach tens automatically
        """
        mean_sum_of_squared_TD = self.get_mean(self.squared_TD_sums_of_episodes) #just for visualization purposes
        self.square_TD_means.append(mean_sum_of_squared_TD.detach().numpy()) #just for visualization purposes
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        self.critic_optimizer.optimize(
            objective=self.get_mean,
            model=self.critic.model,
            objective_input=self.squared_TD_sums_of_episodes)
        
        #############################################
        # YOUR CODE ABOVE
        #############################################
        super().iteration_update()

    def reset_iteration(self):
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        self.squared_TD_sums_of_episodes = []
        
        #############################################
        # YOUR CODE ABOVE
        #############################################
        super().reset_iteration()

<h2 style="color:#A7BD3F;"> Section 5: Testing</h2>

***

As usual, here you have a full freedom of choice: you can tune whatever you want, change whatever you want. 
Play with hyperparameters, choose number of episodes and number of iterations. You may also vary the length of one episode. There is plenty of work. Applying is not that straightforward, we made some necessary stuff but your main objective here is to apply your ML an mathematical intuition to obtain the best result you can. You will definitely have some questions. So do not hesitate to DM me on telegram 😊 -> @odinmaniac

Some addendums:
* The problem is pretty stochastic, so sometimes you can occasionally obtain some good results. But the point here is to achieve a convergence! So, if you didn't obtain a stabilization of parameters and loss, it doesn't count (you will be able to see it on 3-rd and 4-th subplot). So, if you think that you obtained some solid results, make plots please, or ask me to launch your notebook if you struggle with hardware or software issues
* If you're desperate, results are bad and you don't know what to do, you could try the following heuristics:
    * disable the optimization of parameter I (the second one - $w_2$)
    * Change learning rate (sometimes you need to change is really drammatically, so it's okay)
    * Bound your weights

In [158]:
class PipelineInvertedPendulumStudentAC(PipelineInvertedPendulumAC):

    def initialize_models(self):
        print('self.dim_input', self.dim_input)
        print('self.dim_output', self.dim_output)
        self.actor_model = models.ModelGaussianConditional(
            expectation_function=self.safe_controller,
            arg_condition=self.observation_init,
            weights=self.initial_weights,
        )
        self.critic_model = ModelNNStudent(self.dim_output, self.dim_input)
        self.running_objective_model = models.ModelQuadForm(weights=self.R1)

    def initialize_actor_critic(self):
        self.critic = CriticActionValueStudent(
            dim_input=self.dim_input,
            dim_output=self.dim_output,
            data_buffer_size=self.data_buffer_size,
            running_objective=self.running_objective,
            discount_factor=self.discount_factor,
            optimizer=self.critic_optimizer,
            model=self.critic_model,
            sampling_time=self.sampling_time,
        )
        self.actor = ActorProbabilisticEpisodicACStudent(
            self.prediction_horizon,
            self.dim_input,
            self.dim_output,
            self.control_mode,
            self.action_bounds,
            action_init=self.action_init,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            critic=self.critic,
            running_objective=self.running_objective,
            model=self.actor_model,
        )

    def initialize_scenario(self):
        self.scenario = EpisodicScenarioAsyncACStudent(
            system=self.system,
            simulator=self.simulator,
            controller=self.controller,
            actor=self.actor,
            critic=self.critic,
            logger=self.logger,
            datafiles=self.datafiles,
            time_final=self.time_final,
            running_objective=self.running_objective,
            no_print=self.no_print,
            is_log=self.is_log,
            is_playback=self.is_playback,
            N_episodes=self.N_episodes,
            N_iterations=self.N_iterations,
            state_init=self.state_init,
            action_init=self.action_init,
            learning_rate=self.learning_rate
        )

    def execute_pipeline(self, **kwargs):
        """
        Full execution routine
        """
        np.random.seed(42)
        super().execute_pipeline(**kwargs)
    
##### Execution here!!! Full list of kwargs can be seen at 
##### rcognita_framework.pipelines.config_blueprints in the ConfigInvertedPendulum
pipeline = PipelineInvertedPendulumStudentAC()
pipeline.execute_pipeline(
    no_visual=True, 
    is_playback=False, 
    no_print=True,
    time_final=0.5, 
    speedup=150,
    N_episodes=1, 
    N_iterations=1000, 
    learning_rate=1e-9,
    initial_weights=-np.array([1,1,1,1,0])*(10**1), # Set here arbitrary ugly weights
    sampling_time=0.1, # Do not change it!
    is_fixed_critic_weights=True, #### it forbids to enter inside a critic update procedure. Do not change it!
)

self.dim_input 1
self.dim_output 5
End of simulation episode
End of simulation episode
new_weights [-3.45651665e-06  1.35666883e-06 -1.58160707e-05  8.10226413e-06
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.83054782e-06 2.71405030e-07 1.24780163e-05 1.71803284e-06
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 1.17709411e-05 -3.18370770e-06  4.13599523e-05 -1.48127540e-05
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.11739265e-05 5.03022352e-07 4.37920012e-05 1.95109226e-06
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 1.40913807e-05 -3.85769092e-07  6.00431435e-05 -2.48457680e-06
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 1.51312842e-05  1.63352569e-08  6.07096932e-05 -2.92679315e-07
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.28659979e-05 1.33242179e-0

End of simulation episode
End of simulation episode
new_weights [1.50655366e-05 3.14565010e-05 5.33908773e-05 2.79092007e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-2.49765571e-06  3.80160895e-05 -4.29326727e-05  3.23253512e-04
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.85998708e-06 3.73529585e-05 8.69710114e-06 3.19166160e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-2.59471379e-06  4.46225978e-05 -9.44425334e-06  3.69102730e-04
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-1.01912214e-06  4.49404750e-05 -1.02659776e-05  3.72954347e-04
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.44402996e-06 4.44048464e-05 8.20296940e-06 3.69208024e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [4.56922083e-07 4.49505455e-05 7.78260883e-06 3.71188722e-04
 0.00000

End of simulation episode
new_weights [4.07876566e-06 7.40390144e-05 2.01878234e-05 6.09486056e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.17269369e-07 7.78948173e-05 1.63178472e-06 6.32469570e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.11819303e-07 7.79014565e-05 6.94191784e-06 6.30606483e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [7.73431351e-06 7.49779931e-05 3.67160056e-05 6.15042122e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.95249695e-06 7.71948489e-05 8.86110986e-06 6.28655199e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-6.25706467e-06  8.24630711e-05 -2.65284878e-05  6.63992323e-04
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [8.04450703e-06 7.92023738e-05 3.21949366e-05 6.46066756e-04
 0.00000000e+00]
End of simulation episode
E

End of simulation episode
End of simulation episode
new_weights [2.96423229e-06 1.05527451e-04 6.43872767e-06 9.24949627e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.05532975e-06 1.05788984e-04 3.51049759e-06 9.31241610e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-9.06441882e-07  1.07206443e-04  1.77217002e-06  9.37420590e-04
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-7.06113802e-06  1.10821928e-04 -4.05051265e-05  9.61315391e-04
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [3.33622437e-06 1.11298916e-04 6.81283757e-06 9.68512275e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.26155447e-06 1.11780390e-04 8.03307442e-06 9.69419596e-04
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-3.73131781e-06  1.14052601e-04 -2.61493678e-05  9.85154329e-04
  0.00000

End of simulation episode
new_weights [2.17910354e-05 1.36494525e-04 4.92415473e-05 1.24660376e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.76336120e-05 1.38280921e-04 3.69525928e-05 1.25520601e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.17587213e-05 1.40554385e-04 6.78493652e-06 1.27087275e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.70663211e-05 1.38328564e-04 2.03815872e-05 1.26119458e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.73192185e-05 1.38232016e-04 2.39484570e-05 1.25981204e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 1.13849101e-05  1.40321590e-04 -1.16978587e-05  1.27504940e-03
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 1.13600671e-05  1.40370979e-04 -1.74353267e-06  1.27583722e-03
  0.00000000e+00]
End of simulation epis

End of simulation episode
new_weights [2.75960238e-06 1.44448846e-04 9.50409136e-06 1.39886240e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.19747916e-06 1.44588312e-04 1.19412985e-06 1.40160264e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 4.36941235e-06  1.43480279e-04 -9.75185606e-07  1.39900031e-03
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 2.28783590e-07  1.45082367e-04 -1.89128814e-05  1.40853601e-03
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [9.03828993e-07 1.45452827e-04 6.36007275e-06 1.40884332e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [5.12325406e-06 1.43716224e-04 2.80741828e-05 1.39771325e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-2.98367056e-06  1.46834862e-04 -1.35072769e-05  1.41764876e-03
  0.00000000e+00]
End of simulation

End of simulation episode
new_weights [1.10945425e-05 1.69122005e-04 5.27414199e-05 1.64271110e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.28595170e-05 1.68017744e-04 4.57764065e-05 1.64099230e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 2.92443783e-08  1.72747251e-04 -3.64205959e-05  1.67688062e-03
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [9.21173690e-06 1.68793141e-04 3.53568773e-05 1.65555515e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [9.62163192e-06 1.68337895e-04 2.26494025e-05 1.65904441e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-1.09950747e-05  1.75064062e-04 -1.77100274e-04  1.73284749e-03
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [3.60379060e-06 1.73614549e-04 2.13048851e-05 1.72270065e-03
 0.00000000e+00]
End of simulation epis

End of simulation episode
new_weights [4.70901615e-06 2.00359371e-04 8.26260745e-06 1.92643022e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [-4.05068733e-06  2.04013722e-04 -2.67236720e-05  1.95135869e-03
  0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [5.75047934e-07 2.03835805e-04 9.87792774e-06 1.94788604e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.15454855e-05 1.99310098e-04 5.40547492e-05 1.92302871e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.31544535e-05 1.98889779e-04 6.67222831e-05 1.91913597e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [6.48885443e-06 2.01307829e-04 8.12493990e-06 1.94295853e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [ 2.05512899e-06  2.03280831e-04 -1.09174329e-05  1.95469783e-03
  0.00000000e+00]
End of simulation epis

End of simulation episode
End of simulation episode
new_weights [1.43321988e-05 2.32259009e-04 4.28605956e-05 2.26378117e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.56301959e-05 2.31717610e-04 5.02894686e-05 2.26000913e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [1.77927802e-05 2.31029929e-04 6.38352996e-05 2.25485959e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.40832874e-05 2.28118257e-04 7.01017251e-05 2.24482876e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.17847831e-05 2.29215658e-04 6.42766514e-05 2.24991429e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.22722834e-05 2.28970904e-04 6.60095474e-05 2.24865866e-03
 0.00000000e+00]
End of simulation episode
End of simulation episode
new_weights [2.49994563e-05 2.27899634e-04 7.67847674e-05 2.24274083e-03
 0.00000000e+00]
End of

FileNotFoundError: [Errno 2] No such file or directory: './critic_plots/1000-iters_1-episodes_0.5-fintime'

In [ ]:
# import pandas as pd
# import numpy as np
# df = pd.read_csv('states.csv')
# df.drop(['Unnamed: 0'], axis=1).plot()
# df[['0', '1']].plot()
# df = pd.read_csv('actions.csv')
# df['0'].plot()

# Grading

No auto-grading here. Just send me (@odinmaniac) your strong evidence that pendulum is stabilized and i will tell you your grade. Here are some intuition behind the grading.
* Perfect (100 points): pendulum is stabilized, a good optimum was found (episodic mean > -300)
* Good (60-99 points): pendulum is stabilized and, an optimum was found but not that good (episodic mean < -300)
* Acceptable (40-59): everything is converged (plotted TD is about zero, weights are almost stationary during learning) - you're just unlucky. Try another params, another topology, another whatever. PM me if you're completely stuck.
There is no possibility to obtain lower than 40 points if you've submitted in time. But every day of delay will cost you 10 points. After 4-days delay you won't obtain more than 60 points. This is due to the size of this assignment and the end of the term is soon, so just get this done quickly.

In [ ]:
a = np.zeros((6,6))
np.fill_diagonal(a, 3)
a

In [ ]:
for i in range (100):
    print (i % 10 == 7)